# Глубинное обучение для текстовых данных, ФКН ВШЭ

## Домашнее задание 2: Рекуррентные нейронные сети

### Оценивание и штрафы

Максимально допустимая оценка за работу — __10 (+5) баллов__. Сдавать задание после указанного срока сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн: 14.10.24 23:59__   
__Жесткий дедлайн: 17.10.24 23:59__

### О задании

В этом задании вам предстоит самостоятельно реализовать модель LSTM для решения задачи классификации с пересекающимися классами (multi-label classification). Это вид классификации, в которой каждый объект может относиться одновременно к нескольким классам. Такая задача часто возникает при классификации фильмов по жанрам, научных или новостных статей по темам, музыкальных композиций по инструментам и так далее.

В нашем случае мы будем работать с датасетом биотехнических новостей и классифицировать их по темам. Этот датасет уже предобработан: текст приведен к нижнему регистру, удалена пунктуация, все слова разделены проблелом.

In [ ]:
%pip install gdown
!gdown https://drive.google.com/uc?id=1OCbRPznUPXmj9IC410HzL4VldgUhXZCm

In [139]:
import numpy as np
import pandas as pd
import nltk
import re
import random
import os
from nltk.corpus import stopwords
import torch
import wandb
import warnings
from collections import defaultdict
warnings.filterwarnings("ignore")

wandb.login()

device = 'cuda' if torch.cuda.is_available() else 'cpu'

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/bspanfilov/.netrc


In [140]:
def set_random_seed(seed):
    """
    Set random seed for model training or inference.

    Args:
        seed (int): defines which seed to use.
    """
    # fix random seeds for reproducibility
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    # benchmark=True works faster but reproducibility decreases
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

set_random_seed(1)

In [141]:
dataset = pd.read_csv('biotech_news.tsv', sep='\t')
dataset.head()

,text,labels
0,drive your plow over the bones of the dead by ...,other
1,in the recently tabled national budget denel h...,other
2,shares take a break its good for you picture g...,other
3,reso is currently hiring for two positions pro...,other
4,charter buyer club what is the charter buyer c...,other


## Предобработка лейблов


__Задание 1 (1.5 балла)__. Как вы можете заметить, лейблы записаны в виде строк, разделенных запятыми. Для работы с ними нам нужно преобразовать их в числа. Так как каждый объект может принадлежать нескольким классам, закодируйте лейблы в виде векторов из 0 и 1, где 1 означает, что объект принадлежит соответствующему классу, а 0 – не принадлежит. Имея такую кодировку, мы сможем обучить модель, решая задачу бинарной классификации для каждого класса.

In [142]:
labels = set()
for label_lst in dataset.labels.unique():
    for label in label_lst.split(', '):
        labels.add(label)

In [143]:
label_to_num = {v:k for k, v in enumerate(labels)}

In [144]:
def get_labels_array(labels: str):
    target = np.zeros(len(label_to_num))
    for label in labels.split(', '):
        target[label_to_num[label]] = 1

    return target

In [145]:
dataset['num_labels'] = dataset['labels'].apply(get_labels_array)

In [146]:
dataset.head(3)

,text,labels,num_labels
0,drive your plow over the bones of the dead by ...,other,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,in the recently tabled national budget denel h...,other,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,shares take a break its good for you picture g...,other,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


## Предобработка данных

В этом задании мы будем обучать рекуррентные нейронные сети. Как вы знаете, они работают лучше для коротких текстов, так как не очень хорошо улавливают далекие зависимости. Для уменьшение длин текстов их стоит почистить.

Сразу разделим выборку на обучающую и тестовую, чтобы считать все нужные статистики только по обучающей.

In [147]:
from sklearn.model_selection import train_test_split

texts_train, texts_test, y_train, y_test = train_test_split(
    dataset['text'].to_numpy(),
    dataset['num_labels'].to_numpy(),
    test_size=0.2,  # do not change this
    random_state=0  # do not change this
)

In [148]:
new_y_train = np.zeros((len(y_train), len(y_train[0])))

for i in range(len(y_train)):
    for j in range(len(y_train[0])):
        new_y_train[i, j] = y_train[i][j]

y_train = new_y_train

In [149]:
new_y_test = np.zeros((len(y_test), len(y_test[0])))

for i in range(len(y_test)):
    for j in range(len(y_test[0])):
        new_y_test[i, j] = y_test[i][j]

y_test = new_y_test

In [150]:
texts_train.shape, y_train.shape, texts_test.shape, y_test.shape

((2431,), (2431, 29), (608,), (608, 29))

__Задание 2 (1.5 балла)__. Удалите из текстов стоп слова, слишком редкие и слишком частые слова. Гиперпараметры подберите самостоятельно (в идеале их стоит подбирать по качеству на тестовой выборке). Если вы считаете, что стоит добавить еще какую-то обработку, то сделайте это. Важно не удалить ничего, что может повлиять на предсказание класса.

In [151]:
stop_words = stopwords.words('english')

In [152]:
word_to_num = defaultdict(int)
for text in texts_train:
    text = text.split()
    for word in text:
        word_to_num[word] += 1

In [153]:
def remove_stopwords(text):
    clear_texts = []
    clear_text = [word for word in text.split(' ') if word not in stop_words and 7 < word_to_num[word] < 2500]

    return ' '.join(clear_text)

In [154]:
texts_train = np.array([remove_stopwords(text) for text in texts_train])
texts_test = np.array([remove_stopwords(text) for text in texts_test])

In [155]:
texts_train.shape, y_train.shape, texts_test.shape, y_test.shape

((2431,), (2431, 29), (608,), (608, 29))

__Задание 3 (2 балла)__. Осталось перевести тексты в индексы токенов, чтобы их можно было подавать в модель. У вас есть две опции, как это сделать:
1. __(+0 баллов)__ Токенизировать тексты по словам.
2. __(до +5 баллов)__ Реализовать свою токенизацию BPE. Количество баллов будет варьироваться в зависимости от эффективности реализации. При реализации нельзя пользоваться специализированными библиотеками.

Токенизируйте тексты, переведите их в списки индексов и сложите вместе с лейблами в `DataLoader`. Не забудьте добавить в `DataLoader` `collate_fn`, которая будет дополнять все короткие тексты в батче паддингами. Для маппинга токенов в индексы вам может пригодиться `gensim.corpora.dictionary.Dictionary`.

In [156]:
from collections import defaultdict
import time
from IPython.display import clear_output


def word_tokenizer(texts: list[str]) -> list[list]:
    return [text.split(' ') for text in texts]


In [157]:
tokenized_train = word_tokenizer(texts_train)
tokenized_test = word_tokenizer(texts_test)

In [158]:
import gensim
from gensim.corpora.dictionary import Dictionary

dictionary = Dictionary(word_tokenizer(np.concatenate((texts_train, texts_test))))


In [159]:
from torch.utils.data import Dataset

class BaseDataset(Dataset):
    def __init__(
        self,
        texts,
        labels,
        dictionary
    ):

        self.texts = [torch.tensor([dictionary.token2id[word] for word in text]) for text in texts]
        self.labels = labels

    def __getitem__(self, ind):

        instance_data = {
            "text": self.texts[ind],
            "labels": self.labels[ind],
        }

        return instance_data

    def __len__(self):
        return len(self.labels)

In [160]:
train_dataset = BaseDataset(tokenized_train, y_train, dictionary)
test_dataset = BaseDataset(tokenized_test, y_test, dictionary)

In [161]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(dataset_items: list[dict]):

    result_batch = {}

    dataset_items = sorted(dataset_items, key=lambda x: len(x["text"]), reverse=True) # это было лишнее действие, думал получится код совместимым с packed_padded_sequence сделать

    result_batch['lengths'] = [len(sample["text"]) for sample in dataset_items]
    result_batch['texts'] = pad_sequence(
        [sample["text"] for sample in dataset_items], batch_first=True
    )

    result_batch['labels'] = torch.tensor([sample['labels'] for sample in dataset_items])


    return result_batch

In [162]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=True,
    collate_fn=collate_fn,
    pin_memory=True,
    drop_last=True,
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=128,
    shuffle=True,
    collate_fn=collate_fn,
    pin_memory=True,
    drop_last=False,
)

## Метрика качества

Перед тем, как приступить к обучению, нам нужно выбрать метрику оценки качества. Так как в задаче классификации с пересекающимися классами классы часто несбалансированы, чаще всего в качестве метрики берется [F1 score](https://en.wikipedia.org/wiki/F-score).

Функция `compute_f1` принимает истинные метки и предсказанные и считает среднее значение F1 по всем классам. Используйте ее для оценки качества моделей.

$$
F1_{total} = \frac{1}{K} \sum_{k=1}^K F1(Y_k, \hat{Y}_k),
$$
где $Y_k$ – истинные значения для класса k, а $\hat{Y}_k$ – предсказания.

In [163]:
from sklearn.metrics import f1_score

def compute_f1(y_true, y_pred):
    assert y_true.ndim == 2
    assert y_true.shape == y_pred.shape

    return f1_score(y_true, y_pred, average='macro')

## Обучение моделей

### RNN

В качестве бейзлайна обучим самую простую рекуррентную нейронную сеть. Напомним, что блок RNN выглядит таким образом.

<img src="https://i.postimg.cc/yYbNBm6G/tg-image-1635618906.png" alt="drawing" width="400"/>

Его скрытое состояние обновляется по формуле
$h_t = \sigma(W x_{t} + U h_{t-1} + b_h)$. А предсказание считается с помощью применения линейного слоя к последнему токену
$o_T = V h_T + b_o$. В качестве функции активации выберите гиперболический тангенс. 

__Задание 4 (2 балла)__. Реализуйте RNN в соответствии с формулой выше и обучите ее на нашу задачу. Нулевой скрытый вектор инициализируйте нулями, так модель будет обучаться стабильнее, чем при случайной инициализации. После этого замеряйте качество на тестовой выборке. У вас должно получиться значение F1 не меньше 0.33, а само обучение не должно занимать много времени.

In [164]:
from torch import nn
from torch.nn import Sequential


class BaselineRNN(nn.Module):
    def __init__(self, src_vocab_size, input_size, hidden_dim, output_size):
        super().__init__()
        self.hidden_dim = hidden_dim

        self.embedding = nn.Embedding(src_vocab_size, input_size)

        self.W = nn.Linear(input_size, hidden_dim, bias=False)
        self.U = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.b_h = nn.Parameter(torch.zeros(hidden_dim))

        self.V = nn.Linear(hidden_dim, output_size, bias=False)
        self.b_o = nn.Parameter(torch.zeros(output_size))

        self.tanh = nn.Tanh()
        

    def forward(self, texts, lengths, **batch):
        h0 = torch.zeros(texts.shape[0], self.hidden_dim).to(device)

        embedded_text = self.embedding(texts)
        all_outputs = []

        for i in range(texts.shape[1]):
            h0 = self.tanh(self.U(h0) + self.W(embedded_text[:, i, :].squeeze(1)) + self.b_h) # <bs, output_size>
            all_outputs.append(self.V(h0) + self.b_o)
        
        last_outputs = []
        for i, length in enumerate(lengths):
            last_outputs.append(all_outputs[length - 1][i, :])

        return torch.stack(last_outputs)

    def __str__(self):
        """
        Model prints with the number of parameters.
        """
        all_parameters = sum([p.numel() for p in self.parameters()])
        trainable_parameters = sum(
            [p.numel() for p in self.parameters() if p.requires_grad]
        )

        result_info = super().__str__()
        result_info = result_info + f"\nAll parameters: {all_parameters}"
        result_info = result_info + f"\nTrainable parameters: {trainable_parameters}"

        return result_info

In [165]:
from tqdm import tqdm
import torch.nn.functional as F

def train_epoch(net, train_loader, optimizer, lr_scheduler, criterion, wandb):
    losses = []
    possible_thresholds = [0.3, 0.6, 0.9, 0.95]
    f1_scores = {}
    for threshold in possible_thresholds:
        f1_scores[threshold] = []


    for batch in tqdm(train_loader, desc="train", total=len(train_loader)):
        texts = torch.tensor(batch['texts']).to(device)
        lengths = torch.tensor(batch['lengths']).to(device)
        
        optimizer.zero_grad()
        
        out = net(texts, lengths).to('cpu')
        
        loss = criterion(out, batch['labels'])
        torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=5)
        loss.backward()
        losses.append(loss.detach().numpy())

        optimizer.step()
        
        sigm_preds = torch.sigmoid(out)
        for threshold in possible_thresholds:
            f1 = compute_f1(batch['labels'], sigm_preds > threshold)
            f1_scores[threshold].append(f1)

        if lr_scheduler is not None:
            wandb.log({'lr': lr_scheduler.get_lr()[0]})
            lr_scheduler.step()
            
        wandb.log({'train_CEloss': np.mean(losses),})

    wandb.log(
        {f'train_f1_{threshold}': np.mean(f1_score) for threshold, f1_score in f1_scores.items()}
    )

def save_model(wandb, net, epoch, optimizer, lr_scheduler, ):
    arch = type(net).__name__
    state = {
        "arch": arch,
        "epoch": epoch,
        "state_dict": net.state_dict(),
        "optimizer": optimizer.state_dict(),
        "lr_scheduler": lr_scheduler.state_dict(),
    }
    best_path = str("nlp_hw2_model_best.pth")
    torch.save(state, best_path)
    wandb.save(best_path)

def validate(net, test_loader, criterion, best_f1, wandb):
    with torch.no_grad():
        losses = []
        possible_thresholds = [0.3, 0.6, 0.9, 0.95]
        f1_scores = {}
        for threshold in possible_thresholds:
            f1_scores[threshold] = []

        for batch in tqdm(test_loader, desc="test", total=len(test_loader)):
            texts = torch.tensor(batch['texts']).to(device)
            lengths = torch.tensor(batch['lengths']).to(device)
            
            out = net(texts, lengths).to('cpu')
            
            loss = criterion(out, batch['labels'])
            losses.append(loss.detach().numpy())
            
            sigm_preds = torch.sigmoid(out)
            for threshold in possible_thresholds:
                f1 = compute_f1(batch['labels'], sigm_preds > threshold)
                f1_scores[threshold].append(f1)

        wandb.log({
            'test_CEloss': np.mean(losses),
        })
        wandb.log(
            {f'test_f1_{threshold}': np.mean(f1_score) for threshold, f1_score in f1_scores.items()}
        )

        if np.mean(f1_scores[0.6]) > best_f1:
            best_f1 = np.mean(f1_scores[0.6])
            return best_f1, True
        return best_f1, False


def train(net, train_loader, test_loader, optimizer, lr_scheduler, criterion, epochs, wandb):
    print(net)
    best_f1 = 0
    for epoch in range(epochs):

        net.train()
        train_epoch(net, train_loader, optimizer, lr_scheduler, criterion, wandb)

        net.eval()
        best_f1, save = validate(net, test_dataloader, criterion, best_f1, wandb)
        if save:
            print('Saving new best model...')
            save_model(wandb, net, epoch, optimizer, lr_scheduler)


def train_and_validate(wandb, vocab_size, train_dataloader, test_dataloader, model):
    net = model(src_vocab_size=vocab_size, input_size=wandb.config.input_size, hidden_dim=wandb.config.hidden_dim, output_size=29).to(device)
    if torch.cuda.device_count() > 1:
        print('multiple gpus')
        net = nn.DataParallel(net)
    optimizer = torch.optim.AdamW(net.parameters())
    lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=wandb.config.max_lr, pct_start=0.1, anneal_strategy='cos', steps_per_epoch=len(train_dataloader), epochs=wandb.config.epochs)
    criterion = nn.CrossEntropyLoss()

    train(net, train_dataloader, test_dataloader, optimizer, lr_scheduler, criterion, wandb.config.epochs, wandb)

### LSTM

<img src="https://i.postimg.cc/pL5LdmpL/tg-image-2290675322.png" alt="drawing" width="400"/>

Теперь перейдем к более продвинутым рекурренным моделям, а именно LSTM. Из-за дополнительного вектора памяти эта модель должна гораздо лучше улавливать далекие зависимости, что должно напрямую отражаться на качестве.

Параметры блока LSTM обновляются вот так ($\sigma$ означает сигмоиду):
\begin{align}
f_{t} &= \sigma(W_f x_{t} + U_f h_{t-1} + b_f) \\ 
i_{t} &= \sigma(W_i x_{t} + U_i h_{t-1} + b_i) \\
\tilde{c}_{t} &= \tanh(W_c x_{t} + U_c h_{t-1} + b_i) \\
c_{t} &= f_t \odot c_{t-1} + i_t \odot \tilde{c}_t \\
o_{t} &= \sigma(W_t x_{t} + U_t h_{t-1} + b_t) \\
h_t &= o_t \odot \tanh(c_t)
\end{align}

__Задание 5 (2 балла).__ Реализуйте LSTM по описанной схеме. Выберите гиперпараметры LSTM так, чтобы их общее число (без учета слоя эмбеддингов) примерно совпадало с числом параметров обычной RNN, но размерность скрытого слоя была не меньше 64. Так мы будем сравнивать архитектуры максимально независимо. Обучите LSTM до сходимости и сравните качество с RNN на тестовой выборке. Удалось ли получить лучший результат? Как вы можете это объяснить?

In [166]:
from torch import nn
from torch.nn import Sequential


class LSTMNet(nn.Module):
    def __init__(self, src_vocab_size, input_size, hidden_dim, output_size):
        super().__init__()
        self.hidden_dim = hidden_dim

        self.embedding = nn.Embedding(src_vocab_size, input_size)

        self.W_f = nn.Linear(input_size, hidden_dim, bias=False)
        self.U_f = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.b_f = nn.Parameter(torch.zeros(hidden_dim))

        self.W_i = nn.Linear(input_size, hidden_dim, bias=False)
        self.U_i = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.b_i = nn.Parameter(torch.zeros(hidden_dim))

        self.W_c = nn.Linear(input_size, hidden_dim, bias=False)
        self.U_c = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.b_c = nn.Parameter(torch.zeros(hidden_dim))

        self.W_t = nn.Linear(input_size, hidden_dim, bias=False)
        self.U_t = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.b_t = nn.Parameter(torch.zeros(hidden_dim))

        self.V = nn.Linear(hidden_dim, output_size, bias=False)
        self.b_o = nn.Parameter(torch.zeros(output_size))

        self.tanh = nn.Tanh()
        self.sigm = nn.Sigmoid()
        

    def forward(self, texts, lengths, **batch):
        h0 = torch.zeros(texts.shape[0], self.hidden_dim).to(device)
        c0 = torch.zeros(texts.shape[0], self.hidden_dim).to(device)

        embedded_text = self.embedding(texts)
        all_outputs = []

        for i in range(texts.shape[1]):
            x_i = embedded_text[:, i, :].squeeze(1)
            ft = self.sigm(self.W_f(x_i) + self.U_f(h0) + self.b_f)
            it = self.sigm(self.W_i(x_i) + self.U_i(h0) + self.b_i)
            wave_ct = self.tanh(self.W_c(x_i) + self.U_c(h0) + self.b_c)
            c0 = ft * c0 + it * wave_ct
            ot = self.sigm(self.W_t(x_i) + self.U_t(h0) + self.b_t)
            h0 = ot * self.tanh(c0)

            all_outputs.append(self.V(h0) + self.b_o)
        
        last_outputs = []
        for i, length in enumerate(lengths):
            last_outputs.append(all_outputs[length - 1][i, :])

        return torch.stack(last_outputs)

    def __str__(self):
        """
        Model prints with the number of parameters.
        """
        all_parameters = sum([p.numel() for p in self.parameters()])
        trainable_parameters = sum(
            [p.numel() for p in self.parameters() if p.requires_grad]
        )

        result_info = super().__str__()
        result_info = result_info + f"\nAll parameters: {all_parameters}"
        result_info = result_info + f"\nTrainable parameters: {trainable_parameters}"

        return result_info

__Задание 6 (1 балл).__ В этом задании у вас есть две опции на выбор: добавить __двунаправленность__ для LSTM _или_ добавить __многослойность__. Можно сделать и то, и другое, но дополнительных баллов за это мы не дадим, только бесконечный респект. Обе модификации реализуются довольно просто (буквально 4 строчки кода, если вы аккуратно реализовали модель) и дают примерно одинаковый прирост в качестве. Сделайте выводы: стоит ли увеличивать размер модели в несколько раз?

In [184]:
from torch import nn
from torch.nn import Sequential


class LSTMBlock(nn.Module):
    def __init__(self, input_size, hidden_dim, output_size):
        super().__init__()
        self.hidden_dim = hidden_dim

        self.W_f = nn.Linear(input_size, hidden_dim, bias=False)
        self.U_f = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.b_f = nn.Parameter(torch.zeros(hidden_dim))

        self.W_i = nn.Linear(input_size, hidden_dim, bias=False)
        self.U_i = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.b_i = nn.Parameter(torch.zeros(hidden_dim))

        self.W_c = nn.Linear(input_size, hidden_dim, bias=False)
        self.U_c = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.b_c = nn.Parameter(torch.zeros(hidden_dim))

        self.W_t = nn.Linear(input_size, hidden_dim, bias=False)
        self.U_t = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.b_t = nn.Parameter(torch.zeros(hidden_dim))

        self.V = nn.Linear(hidden_dim, output_size, bias=False)
        self.b_o = nn.Parameter(torch.zeros(output_size))

        self.tanh = nn.Tanh()
        self.sigm = nn.Sigmoid()
        

    def forward(self, x, lengths, **batch):
        # x shape is <bs, length, input_dim>
        h0 = torch.zeros(x.shape[0], self.hidden_dim).to(device)
        c0 = torch.zeros(x.shape[0], self.hidden_dim).to(device)

        all_outputs = []

        for i in range(x.shape[1]):
            x_i = x[:, i, :].squeeze(1)
            ft = self.sigm(self.W_f(x_i) + self.U_f(h0) + self.b_f)
            it = self.sigm(self.W_i(x_i) + self.U_i(h0) + self.b_i)
            wave_ct = self.tanh(self.W_c(x_i) + self.U_c(h0) + self.b_c)
            c0 = ft * c0 + it * wave_ct
            ot = self.sigm(self.W_t(x_i) + self.U_t(h0) + self.b_t)
            h0 = ot * self.tanh(c0)

            all_outputs.append(self.V(h0) + self.b_o)
        
        last_outputs = []
        for i, length in enumerate(lengths):
            last_outputs.append(all_outputs[length - 1][i, :])

        return torch.stack(last_outputs), torch.stack(all_outputs).transpose(1, 0)

    def __str__(self):
        """
        Model prints with the number of parameters.
        """
        all_parameters = sum([p.numel() for p in self.parameters()])
        trainable_parameters = sum(
            [p.numel() for p in self.parameters() if p.requires_grad]
        )

        result_info = super().__str__()
        result_info = result_info + f"\nAll parameters: {all_parameters}"
        result_info = result_info + f"\nTrainable parameters: {trainable_parameters}"

        return result_info

In [185]:
class LSTMMultilayer(nn.Module):

    def __init__(self, src_vocab_size, input_size, hidden_dim, output_size, num_layers=2):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstms = nn.ModuleList()
        self.embedding = nn.Embedding(src_vocab_size, input_size)
        self.lstms.append(LSTMBlock(input_size, hidden_dim, hidden_dim if num_layers > 1 else output_size).to(device))
        for i in range(self.num_layers - 1):
            self.lstms.append(LSTMBlock(hidden_dim, hidden_dim, output_size if i == self.num_layers - 2 else hidden_dim).to(device))
        
    def forward(self, x, lengths):
        x = self.embedding(x) 
        for lstm in self.lstms:
            output, x = lstm(x, lengths)
        return output

In [ ]:
wandb.init(
    project="nlp_hw2_rnn",
    name='testing_LSTMMultilayer',

    config={
    "max_lr": 5e-3,
    "input_size": 128,
    "hidden_dim": 256,
    "epochs": 30,
    }
)

train_and_validate(wandb, len(dictionary.token2id), train_dataloader, test_dataloader, LSTMMultilayer)

In [172]:
wandb.finish()